## Explore

This notebook is running in a `docker container` and being accessed from the docker host, `volumes are mapped` which means any changes here will change the file in the rpository

In [1]:
import os
import pandas as pd
os.chdir("..")

In [2]:
from src import create_app
from src.database import db
from src.models.phrase import Phrase
from src.models.language import Language
from src.models.rating import Rating
from src.models.phraseselection import PhraseSelection

In [3]:
app = create_app(test=True)

In [9]:
with app.app_context():
    db.drop_all()

In [4]:
# write data to db
mock_phraseselection_data_1 = {"userid": "1", "phraseid": 1}
mock_phraseselection_data_4 = {"userid": "2", "phraseid": 4}

mock_phrase_data_1 = {"languageid": 2, "userid": "1",
                              "l1": "l1 phrase 1", "l2": "l2 phrase", "category": "restaurant"}
mock_phrase_data_2 = {"languageid": 2, "userid": "1",
                              "l1": "l1 phrase 2", "l2": "l2 phrase", "category": "restaurant"}
mock_phrase_data_3 = {"languageid": 2, "userid": "1",
                              "l1": "l1 phrase 3" , "l2": "l2 phrase", "category": "restaurant"}
mock_phrase_data_4 = {"languageid": 2, "userid": "1",
                              "l1": "l1 phrase 4", "l2": "l2 phrase", "category": "restaurant"}
mock_phrase_data_5 = {"languageid": 2, "userid": "2",
                              "l1": "l1 phrase 4", "l2": "l2 phrase", "category": "restaurant"}

mock_rating_data_1 = {"phraseid": 1, "userid": "1", "rating": 1}
mock_rating_data_2 = {"phraseid": 1, "userid": "2", "rating": 2}
mock_rating_data_3 = {"phraseid": 1, "userid": "3", "rating": 4}
mock_rating_data_4 = {"phraseid": 1, "userid": "4", "rating": 5}
mock_rating_data_5 = {"phraseid": 2, "userid": "5", "rating": 1}
mock_rating_data_6 = {"phraseid": 2, "userid": "4", "rating": 3}
mock_rating_data_7 = {"phraseid": 3, "userid": "4", "rating": 5}
mock_rating_data_8 = {"phraseid": 3, "userid": "4", "rating": 3}


with app.app_context():
    for mock_phrase_selection_data in [mock_phraseselection_data_1, mock_phraseselection_data_4]:
            db.session.add(PhraseSelection(**mock_phrase_selection_data))
            db.session.commit()

    for mock_phrase_data in [mock_phrase_data_1, mock_phrase_data_2, mock_phrase_data_3, mock_phrase_data_4, mock_phrase_data_5]:
        db.session.add(Phrase(**mock_phrase_data))
        db.session.commit()
        
    for mock_rating_data in [mock_rating_data_1, mock_rating_data_2, mock_rating_data_3, 
                             mock_rating_data_4, mock_rating_data_5, mock_rating_data_6, 
                             mock_rating_data_7, mock_rating_data_8]:
        db.session.add(Rating(**mock_rating_data))
        db.session.commit()




In [6]:
with app.app_context():
    phrases = db.session.query(Phrase).all()
    phraseselections = db.session.query(PhraseSelection).all()

In [7]:
[c.toDict() for c in phrases]

[{'phraseid': 1,
  'userid': '1',
  'languageid': 2,
  'l1': 'l1 phrase 1',
  'l2': 'l2 phrase',
  'category': 'restaurant'},
 {'phraseid': 2,
  'userid': '1',
  'languageid': 2,
  'l1': 'l1 phrase 2',
  'l2': 'l2 phrase',
  'category': 'restaurant'},
 {'phraseid': 3,
  'userid': '1',
  'languageid': 2,
  'l1': 'l1 phrase 3',
  'l2': 'l2 phrase',
  'category': 'restaurant'},
 {'phraseid': 4,
  'userid': '1',
  'languageid': 2,
  'l1': 'l1 phrase 4',
  'l2': 'l2 phrase',
  'category': 'restaurant'},
 {'phraseid': 5,
  'userid': '2',
  'languageid': 2,
  'l1': 'l1 phrase 4',
  'l2': 'l2 phrase',
  'category': 'restaurant'},
 {'phraseid': 6,
  'userid': '1',
  'languageid': 2,
  'l1': 'l1 phrase 1',
  'l2': 'l2 phrase',
  'category': 'restaurant'},
 {'phraseid': 7,
  'userid': '1',
  'languageid': 2,
  'l1': 'l1 phrase 2',
  'l2': 'l2 phrase',
  'category': 'restaurant'},
 {'phraseid': 8,
  'userid': '1',
  'languageid': 2,
  'l1': 'l1 phrase 3',
  'l2': 'l2 phrase',
  'category': 'restau

In [8]:
[c.toDict() for c in phraseselections]

[{'phraseselectionid': 1, 'phraseid': 1, 'userid': '1'},
 {'phraseselectionid': 2, 'phraseid': 4, 'userid': '2'},
 {'phraseselectionid': 3, 'phraseid': 1, 'userid': '1'},
 {'phraseselectionid': 4, 'phraseid': 4, 'userid': '2'},
 {'phraseselectionid': 5, 'phraseid': 1, 'userid': '1'},
 {'phraseselectionid': 6, 'phraseid': 4, 'userid': '2'},
 {'phraseselectionid': 7, 'phraseid': 1, 'userid': '1'},
 {'phraseselectionid': 8, 'phraseid': 4, 'userid': '2'}]

In [9]:
with app.app_context():
    phrase_selections_of_user = PhraseSelection.query.filter(PhraseSelection.userid =="1").all()
    phrase_selection_phraseids = [p.phraseid for p in phrase_selections_of_user]
    db_response =db.session.query(Phrase).\
        filter(Phrase.category == "restaurant").\
          filter(Phrase.phraseid.not_in(phrase_selection_phraseids)).all()

In [24]:
phraseids = [1, 2]
with app.app_context():
    db_response = db.session.query(Rating).filter(Rating.phraseid.in_(phraseids)).with_entities(Rating.phraseid, db.func.avg(Rating.rating)).group_by(Rating.phraseid).all()

In [25]:
[c for c in db_response]

[(1, Decimal('3.0000000000000000')), (2, Decimal('2.0000000000000000'))]

In [90]:
type(db)

flask_sqlalchemy.extension.SQLAlchemy

In [98]:
def query(message: str, db) -> pd.DataFrame:
    """
    runs raw sql command and returns data as pandas dataframe

    message: sql query as raw string
    db: must have .session.execute method
    
    """
    db_response = db.session.execute(text(message))
    rows = db_response.fetchall()
    data = pd.DataFrame(rows, columns = list(db_response.keys()))

    return data  

In [99]:
from sqlalchemy import text

with app.app_context():
    # join ratings and phrases
    # groupby average rating
    # return phraseid average_rating
    data = query("""SELECT * 
                    FROM phrases join ratings on phrases.phraseid = ratings.phraseid""", db)
data.head()

,phraseid,userid,languageid,l1,l2,category
0,1,1,2,l1 phrase 1,l2 phrase,restaurant
1,2,1,2,l1 phrase 2,l2 phrase,restaurant
2,3,1,2,l1 phrase 3,l2 phrase,restaurant
3,4,1,2,l1 phrase 4,l2 phrase,restaurant
4,5,2,2,l1 phrase 4,l2 phrase,restaurant
